Part 1: Data Ingestion

In [80]:
import requests
from pathlib import Path
import polars as pl
import duckdb

# Define URLs for required files
taxi_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
zone_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

# Create data/raw directory if it doesn't exist
data_dir = Path("data/raw")
data_dir.mkdir(parents=True, exist_ok=True)

# Defines File paths for downloaded data
taxi_path = data_dir / "yellow_tripdata_2024-01.parquet"
zone_path = data_dir / "taxi_zone_lookup.csv"

# Download Files and write to specified paths
def download_file(url, path):
     if path.exists():
        return
     
     with requests.get(url, stream=True, timeout=30) as r:
         r.raise_for_status()
         with open(path, "wb") as f:
             for chunk in r.iter_content(chunk_size=8192):
                 f.write(chunk)

download_file(taxi_url, taxi_path)
download_file(zone_url, zone_path)
print("\nFiles downloaded successfully!")


Files downloaded successfully!


In [81]:
# Define expected columns
expected_columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", "DOLocationID", 
           "passenger_count", "trip_distance", "fare_amount", "tip_amount", "total_amount",
           "payment_type"]

datetime_columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]

# Load Data with Polars
df = pl.read_parquet(taxi_path, columns=expected_columns)

def validate_data(df):
    # Check for missing columns
    missing_cols = set(expected_columns) - set(df.columns)
    if missing_cols:
        raise Exception(f"Missing expected columns: {missing_cols}")

    # Validate datetime columns
    for col in datetime_columns:
        if not df[col].dtype == pl.Datetime:
            try:
                df = df.with_columns(pl.col(col).cast(pl.Datetime))
            except Exception:
                raise Exception(f"Invalid datetime values detected in column: {col}")
    return df

# Print Row Count and Summary
def print_summary(df):
    print("\n=== Dataset Summary ===")
    print(f"Total rows: {len(df):,}")
    print(f"Shape: {df.shape}")
    print("\nData Validated Successfully!")

df = validate_data(df)
print_summary(df)


=== Dataset Summary ===
Total rows: 2,964,624
Shape: (2964624, 10)

Data Validated Successfully!


Part 2: Data Transformation & Analysis

In [82]:
# Remove rows with nulls
def remove_nulls(df):
    num_rows = df.height

    critical_columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", 
                    "DOLocationID", "fare_amount"]
    
    df = df.drop_nulls(critical_columns)

    removed_nulls = num_rows - df.height
    return df, removed_nulls

# Filter out invalid trips tracking reasons for removal
def filter_trips(df):
    current_rows = df.height

    df = df.filter(pl.col("trip_distance") > 0)
    invalid_distance = current_rows - df.height
    current_rows = df.height

    df = df.filter(pl.col("fare_amount") >= 0)
    negative_fare = current_rows - df.height
    current_rows = df.height

    df = df.filter(pl.col("fare_amount") <= 500)
    exceeding_max = current_rows - df.height

    return df, invalid_distance, negative_fare, exceeding_max

# Filter out trips with dropoff before pickup
def filter_time(df):
    num_rows = df.height

    df = df.filter(pl.col("tpep_dropoff_datetime") >= pl.col("tpep_pickup_datetime"))

    removed_time = num_rows - df.height
    return df, removed_time

# Print summary of removals
def save_and_print(df, total_removed, removed_nulls, invalid_distance, negative_fare, exceeding_max, removed_time):
    print("\n=== Cleaned Dataset Summary ===")
    print(f"Total rows removed: {total_removed:,}")
    print(f"Removed null values: {removed_nulls:,}")
    print(f"Removed invalid distances: {invalid_distance:,}")
    print(f"Removed negative fares: {negative_fare:,}")
    print(f"Removed exceeding $500: {exceeding_max:,}")
    print(f"Removed invalid times: {removed_time:,}")

original_rows = df.height

df, removed_nulls = remove_nulls(df)
df, invalid_distance, negative_fare, exceeding_max = filter_trips(df)
df, removed_time = filter_time(df)

total_removed = original_rows - df.height

save_and_print(df, total_removed, removed_nulls, invalid_distance, negative_fare, exceeding_max, removed_time)


=== Cleaned Dataset Summary ===
Total rows removed: 94,522
Removed null values: 0
Removed invalid distances: 60,371
Removed negative fares: 34,065
Removed exceeding $500: 30
Removed invalid times: 56


From the data cleaning summary above, it can be seen that there contained no null values in the critical columns of the dataset however, invalid distances occupied around 63.9% of rows removed from the dataset. The second-most error prone column was the fare amount where 36.1% of rows were removed due to mostly negative amounts with a samll number of amounts exceeding $500. Finally, there existed only 56 out of the total 94,522 rows that were removed due to invalid pickup and dropoff times.

In [ ]:
# Create derived columns for trip duration, pickup hour, day of week, and trip speed
def create_derived_columns(df):
    df = df.with_columns([
        ((pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime")).dt.total_seconds() / 60)
        .alias("trip_duration_minutes"),

        pl.col("tpep_pickup_datetime").dt.hour().alias("pickup_hour"),

        pl.col("tpep_pickup_datetime").dt.weekday().alias("pickup_day_of_week"),
    ]).with_columns([
        (pl.when(pl.col("trip_duration_minutes") > 0)
         .then(pl.col("trip_distance") / (pl.col("trip_duration_minutes") / 60))
         .otherwise(0)
        ).alias("trip_speed_mph"),
    ])

    return df
        
df = create_derived_columns(df)
print("\nDerived columns created successfully!\n")
print(df.schema)


Derived columns created successfully!

Schema([('tpep_pickup_datetime', Datetime(time_unit='ns', time_zone=None)), ('tpep_dropoff_datetime', Datetime(time_unit='ns', time_zone=None)), ('PULocationID', Int32), ('DOLocationID', Int32), ('passenger_count', Int64), ('trip_distance', Float64), ('fare_amount', Float64), ('tip_amount', Float64), ('total_amount', Float64), ('payment_type', Int64), ('trip_duration_minutes', Float64), ('pickup_hour', Int8), ('pickup_day_of_week', Int8), ('trip_speed_mph', Float64)])


In [ ]:
# Create data/clean directory and define output path
data_dir = Path("data/clean")
data_dir.mkdir(parents=True, exist_ok=True)

output_path = data_dir / "yellow_tripdata_2024-01_clean.parquet"

# Save cleaned data file
df.write_parquet(output_path)

print("\nCleaned data saved successfully!")

In [85]:
# Create a DuckDB connection 
con = duckdb.connect()

# Load the zones data into a Polars DataFrame
zones = pl.read_csv("data/raw/taxi_zone_lookup.csv")

# Register the Polars DataFrames as a DuckDB table
con.register("trips", df.to_arrow())
con.register("zones", zones.to_arrow())

The following query shows the top 10 busiest zones by their total number of trips including that zone in descending order.

In [86]:
busiest_pickup_zones = con.execute("""
    SELECT
        z.Zone,
        COUNT(*) AS total_trips
    FROM trips t
    JOIN zones z
    ON t.PULocationID = z.LocationID
    GROUP BY z.Zone
    ORDER BY total_trips DESC
    LIMIT 10;
    """).fetchdf()

print(busiest_pickup_zones)

                           Zone  total_trips
0                Midtown Center       140161
1         Upper East Side South       140134
2                   JFK Airport       138478
3         Upper East Side North       133975
4                  Midtown East       104356
5     Times Sq/Theatre District       102972
6  Penn Station/Madison Sq West       102161
7           Lincoln Square East       101800
8             LaGuardia Airport        87715
9         Upper West Side South        86475


The following query shows the average fare amount paid for trips at every pickup hour of the day ordered by the hour.

In [87]:
avg_fare_hourly = con.execute("""
    SELECT
        pickup_hour,
        AVG(fare_amount) AS avg_fare
    FROM trips
    GROUP BY pickup_hour
    ORDER BY pickup_hour;
    """).fetchdf()

print(avg_fare_hourly)

    pickup_hour   avg_fare
0             0  19.679250
1             1  17.732032
2             2  16.621723
3             3  18.530033
4             4  23.435229
5             5  27.492713
6             6  22.026585
7             7  18.749879
8             8  17.822939
9             9  17.943989
10           10  18.047523
11           11  17.628112
12           12  17.796520
13           13  18.418805
14           14  19.271523
15           15  19.110366
16           16  19.457290
17           17  18.118545
18           18  17.013712
19           19  17.626564
20           20  18.050403
21           21  18.292862
22           22  19.110051
23           23  20.243498


The following query shows the percentage of total trips that were paid for by each payment type available descending by percentage.

In [88]:
trips_by_payment = con.execute("""
    SELECT
        payment_type,
        COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS percentage
    FROM trips
    GROUP BY payment_type
    ORDER BY percentage DESC;
    """).fetchdf()

print(trips_by_payment)

   payment_type  percentage
0             1   80.081196
1             2   14.735400
2             0    4.015363
3             4    0.797045
4             3    0.370997


The following query shows the average tip amount for each day of the week as percentage for only payments made with a credit card ordered by the day of the week.

In [89]:
tip_percentage_card = con.execute("""
    SELECT
        pickup_day_of_week,
        AVG(CASE WHEN fare_amount > 0 THEN tip_amount / fare_amount END) * 100 AS avg_tip_percentage
    FROM trips
    WHERE payment_type = 1
    GROUP BY pickup_day_of_week
    ORDER BY pickup_day_of_week;
    """).fetchdf()

print(tip_percentage_card)

   pickup_day_of_week  avg_tip_percentage
0                   1           25.513977
1                   2           25.729989
2                   3           25.706582
3                   4           29.734458
4                   5           25.595719
5                   6           26.293897
6                   7           25.100984


The following query shows the top 5 most common pickup and dropoff zone pairs by the total amount of trips including both zones in descending order of trips.

In [90]:
common_trip_routes = con.execute("""
    SELECT
        zp.Zone AS pickup_zone,
        zd.Zone AS dropoff_zone,
        COUNT(*) AS trip_count
    FROM trips t
    JOIN zones zp
        ON t.PULocationID = zp.LocationID
    JOIN zones zd
        ON t.DOLocationID = zd.LocationID
    GROUP BY pickup_zone, dropoff_zone
    ORDER BY trip_count DESC
    LIMIT 5;
    """).fetchdf()

print(common_trip_routes)

             pickup_zone           dropoff_zone  trip_count
0  Upper East Side South  Upper East Side North       21642
1  Upper East Side North  Upper East Side South       19199
2  Upper East Side North  Upper East Side North       15200
3  Upper East Side South  Upper East Side South       14119
4         Midtown Center  Upper East Side South       10139


Part 3: Dashboard Development